In [12]:
import torch

import zipfile
from pathlib import Path

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
import datetime
import random
import cv2
import src.models as mdls
import src.dataloaders as dataloaders
import src.utils as utils

In [13]:
config = utils.get_options()

## for google colab

In [14]:
model = mdls.get_pretrained_VGG16()
dataloader = dataloaders.dataloader_for_detection

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [16]:
# Define a directory to save the checkpoints
checkpoint_dir = "checkpoints/"

# Ensure the directory exists
import os
os.makedirs(checkpoint_dir, exist_ok=True)

In [17]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.to(device)
loss_fn.to(device)


MSELoss()

In [18]:
epochs = config['epochs']

for epoch in range(epochs):
    epoch_loss = 0.0
    for sample in (pbar := tqdm(dataloader)):

        img, box = sample[0], sample[1]
        img = img.to(torch.float32)
        box = box.to(torch.float32)
        optimizer.zero_grad()
        pred = model(img)

        loss = loss_fn(pred, box)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}\tLoss: {epoch_loss / len(dataloader)}")
    checkpoint_filename = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

  0%|          | 1/210 [00:29<1:42:54, 29.54s/it]


KeyboardInterrupt: 